In [199]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [200]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [201]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 16

embeddings = CustomEmbedding(vocab_size = vocab_size + 3, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[ 1.0288, -1.0102,  0.5830,  1.2201, -0.1457, -0.4269, -0.6493,  0.3485,
          1.8027,  0.9619,  0.0663,  0.0654, -1.1997, -0.1436, -1.5996,  0.7300],
        [ 0.1213, -0.2077, -1.1364,  0.9151,  0.1011,  1.6786,  0.2054, -2.0026,
         -1.6788,  0.9964,  0.3421,  0.1494, -0.2632, -0.7698,  0.3498,  0.8126]],
       grad_fn=<EmbeddingBackward0>)


In [202]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 2
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

rand_ds = RandomIntegerDataset(2, 10, 10, [i for i in range(10)])
dataloader = DataLoader(rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [203]:
input, label = next(iter(dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[2, 3, 4, 4, 2, 0, 2, 0, 6],
        [8, 7, 2, 4, 0, 9, 6, 8, 5]])
tensor([[11,  0,  0,  2,  2,  2,  3,  4,  4,  6],
        [11,  0,  2,  4,  5,  6,  7,  8,  8,  9]])
tensor([[ 0,  0,  2,  2,  2,  3,  4,  4,  6, 12],
        [ 0,  2,  4,  5,  6,  7,  8,  8,  9, 12]])


In [204]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [205]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer):
    
    model.train()

    epoch_loss = 0.0

    for i, ((source, target), label) in enumerate(dataloader):
        print(f'Processing Batch {i+1}...')

        # FORWARD
        pred = model(source, target)
        batch_loss = loss_fn(pred, label)

        # LOG
        epoch_loss += batch_loss.item()

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / len(dataloader.dataset)

    return average_epoch_loss

def run_test_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    
    model.eval()

In [206]:
EPOCHS = 1

training_losses = list()

for i in range(EPOCHS):

    training_loss = run_train_epoch(dataloader, model, loss_fn, optim)
    training_losses.append(training_loss)

Processing Batch 1...


AssertionError: 